# iTEBD for Infinite Heisenberg Chain

This notebook demonstrates the infinite Time-Evolving Block Decimation (iTEBD) algorithm in Vidal's Γ-Λ canonical form for computing the ground state energy of an infinite Heisenberg spin chain.

In [1]:
# Import necessary libraries
import sys
sys.path.insert(0, '..')

import numpy as np
# import matplotlib.pyplot as plt
from bench.inf_tebd import inf_tebd_spin

## Run iTEBD with Default Parameters

For spin-1/2 Heisenberg chain, the exact ground state energy per site is:
$$E_{\text{exact}} = \frac{1}{4} - \ln(2) \approx -0.443147$$

In [ ]:
# Run iTEBD with moderate parameters
E_final, E_history, E_exact = inf_tebd_spin(
    dt_list=[0.1, 0.01, 0.001, 0.0001],
    Nkeep=100,
    J=1.0,
    spin=0.5,
    tol=1e-10,
    max_iter=1000,
    verbose=True
)

## Energy Convergence Plot

In [ ]:
# Plot energy convergence
plt.figure(figsize=(12, 5))

# Plot 1: Energy vs iteration
plt.subplot(1, 2, 1)
plt.plot(E_history, 'b-', linewidth=1, alpha=0.7)
plt.axhline(y=E_exact, color='r', linestyle='--', label=f'Exact: {E_exact:.6f}')
plt.xlabel('Iteration')
plt.ylabel('Energy per site')
plt.title('iTEBD Energy Convergence')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Error vs iteration (log scale)
plt.subplot(1, 2, 2)
error = np.abs(E_history - E_exact)
plt.semilogy(error, 'b-', linewidth=1, alpha=0.7)
plt.xlabel('Iteration')
plt.ylabel('|E - E_exact|')
plt.title('Absolute Error (log scale)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal energy: {E_final:.10f}")
print(f"Exact energy: {E_exact:.10f}")
print(f"Final error: {abs(E_final - E_exact):.2e}")

## Test Different Bond Dimensions

In [ ]:
# Test convergence with different bond dimensions
Nkeep_list = [20, 40, 60, 80, 100, 150]
final_energies = []
errors = []

for Nkeep in Nkeep_list:
    print(f"\nRunning with Nkeep = {Nkeep}...")
    E_final, _, E_exact = inf_tebd_spin(
        dt_list=[0.1, 0.01, 0.001],
        Nkeep=Nkeep,
        verbose=False
    )
    final_energies.append(E_final)
    errors.append(abs(E_final - E_exact))
    print(f"  E = {E_final:.10f}, Error = {errors[-1]:.2e}")

# Plot results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(Nkeep_list, final_energies, 'bo-', label='iTEBD')
plt.axhline(y=E_exact, color='r', linestyle='--', label=f'Exact: {E_exact:.6f}')
plt.xlabel('Bond Dimension (Nkeep)')
plt.ylabel('Energy per site')
plt.title('Energy vs Bond Dimension')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.semilogy(Nkeep_list, errors, 'ro-')
plt.xlabel('Bond Dimension (Nkeep)')
plt.ylabel('|E - E_exact|')
plt.title('Error vs Bond Dimension (log scale)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Performance Profiling with Scalene

Profile the iTEBD function to identify performance bottlenecks.

In [2]:
# Load scalene extension
%load_ext scalene

Scalene extension successfully loaded. Note: Scalene currently only
supports CPU+GPU profiling inside Jupyter notebooks. For full Scalene
profiling, use the command line version. To profile in line mode, use
`%scrun [options] statement`. To profile in cell mode, use `%%scalene
[options]` followed by your code.

NOTE: in Jupyter notebook on MacOS, Scalene cannot profile child
processes. Do not run to try Scalene with multiprocessing in Jupyter
Notebook.


In [5]:
# Profile with moderate parameters
%scrun inf_tebd_spin(spin=1, dt_list=[0.1, 0.01, 0.001, 0.0001], Nkeep=1000, max_iter=1000, verbose=True)

[23:47:38] Starting iTEBD: spin=1, J=1.0, Nkeep=1000
[23:47:38] Time steps: [0.1, 0.01, 0.001, 0.0001]
[23:47:38] Time step dt = 0.1
[23:47:43]   Iter 100/1000: E = -1.3454095265, dE = 3.57e-06, χ = (120, 120)
[23:47:46]   Iter 200/1000: E = -1.3454514751, dE = 9.82e-10, χ = (124, 120)
[23:47:47]   Converged at iteration 228: E = -1.3454514854, dE = 9.88e-11
[23:47:47] Time step dt = 0.01
[23:47:51]   Iter 100/1000: E = -1.3939194628, dE = 6.23e-05, χ = (96, 100)
[23:47:54]   Iter 200/1000: E = -1.3959515193, dE = 4.15e-06, χ = (96, 100)
[23:47:58]   Iter 300/1000: E = -1.3961083674, dE = 4.18e-07, χ = (96, 100)
[23:48:01]   Iter 400/1000: E = -1.3961257166, dE = 5.40e-08, χ = (96, 100)
[23:48:05]   Iter 500/1000: E = -1.3961280957, dE = 8.18e-09, χ = (96, 100)
[23:48:08]   Iter 600/1000: E = -1.3961284722, dE = 1.39e-09, χ = (96, 100)
[23:48:12]   Iter 700/1000: E = -1.3961285386, dE = 2.60e-10, χ = (96, 100)
[23:48:14]   Converged at iteration 760: E = -1.3961285485, dE = 9.92e-11
[2


Scalene: profile saved to scalene-profile.json
  To view in browser:  scalene view
  To view in terminal: scalene view --cli


## Comparison: iTEBD vs Iterative Diagonalization

Compare the results and computational efficiency of iTEBD with the iterative diagonalization approach.

In [ ]:
from bench.iter_diag import iter_diag_spin
import time

# Run iTEBD
t0 = time.time()
E_tebd, _, E_exact = inf_tebd_spin(
    dt_list=[0.1, 0.01, 0.001],
    Nkeep=100,
    verbose=False
)
time_tebd = time.time() - t0

# Run iterative diagonalization
t0 = time.time()
_, EG_iter, _ = iter_diag_spin(
    N=100,
    Nkeep=100,
    verbose=False
)
E_iter = EG_iter[-1]
time_iter = time.time() - t0

print("\n" + "="*60)
print("Comparison: iTEBD vs Iterative Diagonalization")
print("="*60)
print(f"Exact energy:              {E_exact:.10f}")
print(f"\niTEBD result:              {E_tebd:.10f}")
print(f"iTEBD error:               {abs(E_tebd - E_exact):.2e}")
print(f"iTEBD time:                {time_tebd:.2f} seconds")
print(f"\nIter Diag result:          {E_iter:.10f}")
print(f"Iter Diag error:           {abs(E_iter - E_exact):.2e}")
print(f"Iter Diag time:            {time_iter:.2f} seconds")
print(f"\nSpeedup: {time_iter / time_tebd:.2f}x")
print("="*60)